In [1]:
import os
import time
import csv
import numpy as np
import pandas as pd

import sklearn.metrics
import re
import unicodedata
from tqdm import tqdm
import sys
import hashlib
import random


Candidate String Generation

In [2]:


import re
df = pd.read_csv("crawled_issue/data.csv")
df_unique = df.drop_duplicates(subset='Issue ID', keep='first')
df = df_unique
print(df.head())


   Issue ID                                              Title  \
0     11164                              Create testdriver.yml   
1     11163  [tests] Add hydrogen-2023 to static-build test...   
2     11162          [remix] Update `@remix-run/dev` to v2.6.0   
3     11161                    [tests] Add astro to dependabot   
4     11160  [framework-fixtures]: Bump the core group in /...   

                                          Issue Body             Closed At  
0                                EDIT" please delete  2024-02-15T21:51:06Z  
1    Add hydrogen-2023 to static-build test fixtures  2024-02-15T20:59:00Z  
2  This auto-generated PR updates `@remix-run/dev...  2024-02-15T19:32:15Z  
3                            Add astro to dependabot  2024-02-15T18:41:03Z  
4  Bumps the core group in /packages/static-build...  2024-02-15T17:25:58Z  


In [3]:
# !pip3 install openpyxl 
excel_data = pd.read_excel('dataset/Secret-Regular-Expression.xlsx')

# Read the values of the file in the dataframe
regex = pd.DataFrame(excel_data, columns=[
'Pattern_ID','Secret Type',	'Regular Expression','Source'])

In [4]:

data_dict={}
for j in df.index:
        # if df["id"][j] != "1165939311":
        #         continue
        input_string =    str(df["Issue Body"][j])    
        input_string = re.sub(r'[\'"\│]', '', input_string)
        dir_list_clean = re.sub(r'drwx[-\s]*\d+\s+\w+\s+\w+\s+\d+\s+\w+\s+\d+\s+[0-9a-fA-F-]+.*','',input_string)
        shell_code_free_text = re.sub(r'```shell([^`]+)```','',dir_list_clean,flags=re.IGNORECASE)
        shell_code_free_text = re.sub(r'```Shell\s*"([^"]*)"\s*```','',shell_code_free_text,flags=re.IGNORECASE)
        # saved_game_free_text = re.sub(r'```([^`]+)```','',shell_code_free_text) #etay jhamela hobe
        saved_game_free_text = re.sub(r'<details><summary>Saved game</summary>\n\n```(.*?)```', '', shell_code_free_text)
        remove_packages = re.sub(r'(\w+\.)+\w+','',saved_game_free_text)
        java_exp_free_text = re.sub(r'at\s[\w.$]+\.([\w]+)\(([^:]+:\d+)\)','',remove_packages)
        # url_free_text= re.sub(https?://[^\s#]+#[A-Za-z0-9\-]+,'', java_exp_free_text, flags=re.IGNORECASE)
        url_with_fragment_text= re.sub(r'https?://[^\s#]+#[A-Za-z0-9\-\=\+]+','', java_exp_free_text, flags=re.IGNORECASE)
        url_free_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',url_with_fragment_text)
        commit_free_text= re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', url_free_text, flags=re.IGNORECASE)
        file_path_free_text = re.sub(r"/[\w/. :-]+",'',commit_free_text)
        file_path_free_text = re.sub( r'(/[^/\s]+)+','',file_path_free_text)
        sha256_free_text = re.sub(r'sha256\s*[:]?[=]?\s*[a-fA-F0-9]{64}','',file_path_free_text)
        sha1_free_text = re.sub(r'git-tree-sha1\s*=\s*[a-fA-F0-9]+','',sha256_free_text)
        build_id_free_text = re.sub(r'build-id\s*[:]?[=]?\s*([a-fA-F0-9]+)','',sha1_free_text)
        guids_free_text = re.sub(r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)','',build_id_free_text)
        uuids_free_text = re.sub(r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)','',guids_free_text)
        event_id_free_text = re.sub(r'<([^>]+)>','',uuids_free_text)
        UUID_free_text = re.sub(r'(?:UUID|GUID|version|id)[\\=:"\'\s]*\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
,'',event_id_free_text,flags=re.IGNORECASE) ##without the prefix so many false positives can be omitted
        hex_free_text = re.sub(r'(?:data|address|id)[\\=:"\'\s]*\b0x[0-9a-fA-F]+\b','',UUID_free_text,flags=re.IGNORECASE) ## deleting hex ids directly can cause issues
        ss_free_text = re.sub(r'Screenshot_(\d{4}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\w+)','',hex_free_text,flags=re.IGNORECASE)
        cleaned_text = ss_free_text
        # file_path = "output.txt"

        # with open(file_path, 'w') as file:
        #                 file.write(cleaned_text)
        data_dict[j] = {'Issue ID':df['Issue ID'][j],'Issue Body':cleaned_text}
        # idx = idx+1
    


cleaned_text_data = pd.DataFrame.from_dict(data_dict, "index")
cleaned_text_data

,Issue ID,Issue Body
0,11164,EDIT please delete
1,11163,Add hydrogen-2023 to static-build test fixtures
2,11162,This auto-generated PR updates `@remix-run` to...
3,11161,Add astro to dependabot
4,11160,Bumps the core group in \n\n| Package | From |...
...,...,...
6904,5,It should display a friendlier error\n
6905,4,nan
6906,3,`now -d list`\n\nshould result in\n\n`now list...
6907,2,"Right now if a user specifies `main` in ``, we..."


In [5]:
###recovered
    
idx = 0
data_dict={}
# start = iter*100000
# end = (iter+1)*100000
for i in regex.index:
    #print(i,regex['Secret Type'][i]) #, regex['Regular Expression'][i])
    # if i%100==0:
    #     print("checkpoint")
    p = re.compile(regex['Regular Expression'][i])
    
    # print("=====================================================================")
    
    for j in df.index:
        
        cleaned_text = cleaned_text_data.loc[j, 'Issue Body']
            # Now you can use 'cleaned_text' for further processing
       
        matches = re.findall(p,cleaned_text)
        for match in set(matches):
                data_dict[idx] = {'Type': regex['Secret Type'][i], 'Issue ID':df['Issue ID'][j],'Candidate String':match} #,'Entropy':shannon_entropy(match)}
                idx = idx+1
    


data = pd.DataFrame.from_dict(data_dict, "index")
data=data.drop_duplicates(subset=["Issue ID", "Candidate String"], keep='first')
print(data.shape)
data.to_csv('crawled_issue/issues-with-candidate-strings.csv')

C:\Users\ahmed\AppData\Local\Temp\ipykernel_3384\2071773696.py:11: FutureWarning: Possible nested set at position 35
  p = re.compile(regex['Regular Expression'][i])
C:\Users\ahmed\AppData\Local\Temp\ipykernel_3384\2071773696.py:11: DeprecationWarning: Flags not at the start of the expression '/(?i)-----\\s*?BEGIN[' (truncated) but at position 1
  p = re.compile(regex['Regular Expression'][i])


(196, 3)


Context Window and Evaluation

In [6]:
data = data.rename(columns={'Issue ID': 'Issue_id'})
print(data.shape)
print(data.head())
merged_df = df.merge(data, left_on='Issue ID', right_on='Issue_id')
print(merged_df.shape)
columns_to_remove = ['Issue_id']
merged_df.drop(columns=columns_to_remove, inplace=True)
print(merged_df.columns)

(196, 3)
           Type  Issue_id                      Candidate String
0   Alibaba API       487        Ahohohhohohohohohohohohohohooh
1  Anypoint API      9055  064746f1-10e5-4262-b32e-09504ff56574
2  Anypoint API      7287  f7284007-d328-48a7-b4e3-ecf669500227
3  Anypoint API      6290  b91e714d-18f8-44e6-9db6-98f1e58cbb5c
4  Anypoint API      6290  f82a0b00-be53-11eb-872d-4aa5adcd539d
(196, 7)
Index(['Issue ID', 'Title', 'Issue Body', 'Closed At', 'Type',
       'Candidate String'],
      dtype='object')


In [7]:
def create_context_window(text, target_string, window_size=200):

    target_index = text.find(target_string)
    #print(target_index)

    if target_index != -1:
        start_index = max(0, target_index - window_size)
        end_index = min(len(text), target_index + len(target_string) + window_size)
        context_window = text[start_index:end_index]
        return context_window

    return None

# Apply the create_context_window function to each row in the DataFrame
merged_df['modified_text'] = merged_df.apply(lambda row: create_context_window(row['Issue Body'], row['Candidate String']), axis=1)
print(merged_df.shape)
print(merged_df.head())

(196, 7)
   Issue ID                                              Title  \
0     10988  Response to preflight request doesn't pass acc...   
1     10988  Response to preflight request doesn't pass acc...   
2     10412                    swagger does not load on vercel   
3     10412                    swagger does not load on vercel   
4     10350          Trek/vccli 825 investigate flakey tests 2   

                                          Issue Body             Closed At  \
0  ### Discussed in https://github.com/vercel/ver...  2023-12-21T16:56:36Z   
1  ### Discussed in https://github.com/vercel/ver...  2023-12-21T16:56:36Z   
2  Hi, yesterday I could load my swagger docs usi...  2023-08-30T07:08:43Z   
3  Hi, yesterday I could load my swagger docs usi...  2023-08-30T07:08:43Z   
4  Seeing if the failure re `patrickedqvist/wait-...  2023-08-15T18:42:59Z   

              Type                          Candidate String  \
0  Generic Pattern                          X-Requested-With 

In [8]:
count=0
inverse_count=0

for i in range(merged_df.shape[0]):
  #print(i)
  main_string=merged_df['Issue Body'][i]
  substring=merged_df['Candidate String'][i]
  #print(main_string.find(substring))
  if main_string.find(substring)!=-1:
    count+=1
  else:
    inverse_count+=1
print("Mismatch: "+str(inverse_count))
print("Ok: "+str(count))

Mismatch: 16
Ok: 180


In [9]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

c:\Users\ahmed\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
X_issue_ids = merged_df['Issue ID'].tolist()
X_text_test = merged_df['Issue Body'].tolist()  # Convert the 'text' column to a list of strings
X_candidate_test = merged_df['Candidate String'].tolist()  # Convert the 'candidate_string' column to a list of strings

In [11]:
def encode_texts(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    return encodings

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [12]:
# Define a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, text_encodings, candidate_encodings, labels):
        self.text_encodings = text_encodings
        self.candidate_encodings = candidate_encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx): #it works fine for training
        text_input_ids = self.text_encodings['input_ids'][idx]
        text_attention_mask = self.text_encodings['attention_mask'][idx]
        candidate_input_ids = self.candidate_encodings['input_ids'][idx]
        candidate_attention_mask = self.candidate_encodings['attention_mask'][idx]
        label = torch.tensor(self.labels[idx])

        return text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, label

In [13]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model_path = "models/adamW_cntxt200_data25k_pre.pth"
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode for inference

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [14]:
text_body_encodings_test = encode_texts(X_text_test)
candidate_encodings_test = encode_texts(X_candidate_test)

In [15]:
print(len(X_text_test))
Y_labels = [0] * len(X_text_test)
Y = np.array(Y_labels)
Y_ =Y.astype(int)
print(Y_)

196
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]


In [16]:
test_dataset = CustomDataset(text_body_encodings_test, candidate_encodings_test, Y_)

In [17]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [18]:
model.to(device)
model.eval()  # Set the model in evaluation mode
c=0
predicted_labels_list = []
with torch.no_grad():

    for batch in test_loader:
        print("Batch %d"%c)
        c+=1

        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = batch

        # Move tensors to the device
        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = (
            text_input_ids.to(device),
            text_attention_mask.to(device),
            candidate_input_ids.to(device),
            candidate_attention_mask.to(device),
            labels.to(device)
        )

        # Perform inference

        outputs = model(input_ids=text_input_ids.type(torch.LongTensor).cuda(), attention_mask=text_attention_mask.type(torch.LongTensor).cuda())
        predicted_labels = torch.argmax(outputs.logits, dim=1)

        # print(f"predicted_labels: {predicted_labels}")
        predicted_labels_list.append(predicted_labels[0])

Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93
Batch 94
Batch 95
Batch 96
Batch 97
Batch 98
Batch 99
Batch 100
Batch 101
Batch 102
Batch 103
Batch 104
Batch 105
Batch 106
Batch 107
Batch 108
Batch 109
Batch 110


In [19]:
predicted_labels_list_output = [f.cpu().numpy().tolist() for f in predicted_labels_list]
print(predicted_labels_list_output)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]


In [20]:

secret_indexes = []
for i in range(len(predicted_labels_list_output)):
    if predicted_labels_list_output[i] == 1 :
        secret_indexes.append(X_issue_ids[i])
secret_indexes = list(set(secret_indexes))
print(secret_indexes)
print(len(secret_indexes))

[178, 1906, 122]
3
